In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from dfcx_scrapi.core.intents import Intents
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions

creds_path = '<YOUR_CREDS_PATH_HERE>'
dev_agent = '<YOUR_AGENT_ID_HERE>'

In [ ]:
# Instanstiate Classes
intents = Intents(creds_path)
cxtools = DataframeFunctions(creds_path)

# Sheets to CX

In [ ]:
df = cxtools.sheets_to_dataframe('<YOUR_SHEET_NAME_HERE>','<YOUR_SHEET_TAB_NAME_HERE>')

In [ ]:
df

In [ ]:
cxtools.bulk_create_intent_from_dataframe(dev_agent, df, update_flag=True, rate_limiter=1)

# CX to Sheets

In [ ]:
# Get all of the intents from the Agent
dev_intents = intents.list_intents(dev_agent)

print('Variable is of type \'{}\''.format(type(dev_intents)))
print('Total # of Intents = {}'.format(len(dev_intents)), '\n')

In [ ]:
# Transform all of the intents into a Dataframe
df_voice = intents.bulk_intent_to_df(dev_agent)

print('Variable is of type \'{}\''.format(type(df_voice)))
print('Total # of Intents = {}'.format(df_voice.intent.nunique()), '\n')

In [ ]:
# Explore the Intents/Training Phrases in your Dataframe
df_voice[df_voice.intent == '<YOUR_INTENT_DISPLAY_NAME_HERE>']

In [ ]:
cxtools.dataframe_to_sheets('<YOUR_SHEET_NAME_HERE>','<YOUR_SHEET_TAB_NAME_HERE>', df_voice)

In [ ]:
final_df = cxtools.sheets_to_dataframe('<YOUR_SHEET_NAME_HERE>','<YOUR_SHEET_TAB_NAME_HERE>')

In [ ]:
cxtools.bulk_create_intent_from_dataframe(dev_agent, final_df, update_flag=True, rate_limiter=1)

# CX to Sheets - Filterd by Intents in Scope of a Flow

In [ ]:
from dfcx_scrapi.tools.copy_util import CopyUtil
from dfcx_scrapi.tools.dataframe_functions import DataframeFunctions
import pandas as pd

dev_creds_path = '<YOUR_CREDS_PATH_HERE>'
voice_agent = '<YOUR_AGENT_ID_HERE>'

cu = CopyUtil(creds_path=dev_creds_path, agent_id=voice_agent)
cxtools = DataframeFunctions(dev_creds_path)

flow = '<YOUR_FLOW_DISPLAY_NAME>'

flow_map = cu.flows.get_flows_map(reverse=True)
pages = cu.pages.list_pages(flow_map[flow])
resources = cu.get_page_dependencies(pages)

for key in resources.keys():
    if key == 'intents':
        intent_list = list(resources[key])
        
all_intents = cu.intents.list_intents()

final_intents = []
for intent in all_intents:
    if intent.name in intent_list:
        final_intents.append(intent)

df = pd.DataFrame()

for intent in final_intents:
    df = df.append(cu.intents.intent_proto_to_dataframe(intent))

# Create the GSheet
cxtools.dataframe_to_sheets('<YOUR_SHEET_NAME_HERE>', '<YOUR_SHEET_TAB_NAME_HERE>', df)